In [14]:
from flask import Flask, render_template, request, redirect
import pickle
import re
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import os
import joblib

In [15]:
#Creat an app object using the flask class
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads/'

In [16]:
# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
portStemmer = PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\OMEN\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\OMEN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
# Load models and vectorizer once
xgboost_model = joblib.load("models/xgboost_model.pkl")
random_forest_model = joblib.load("models/random_forest_model.pkl")
lightgbm_model = joblib.load("models/light_gbm_model.pkl")
logistic_regression_model = joblib.load("models/logistic_regression_model.pkl")
vectorizer = joblib.load("models/vectorizer.pkl")


In [18]:
models = {
    "XGBoost": xgboost_model,
    "Random Forest": random_forest_model,
    "LightGBM": lightgbm_model,
    "Logistic Regression": logistic_regression_model
}

In [19]:
def clean(text):
    text = text.lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

def preprocess(text):
    text = clean(text)
    tokens = [portStemmer.stem(word) for word in text.split() if word not in stop_words]
    return ' '.join(tokens)

In [20]:
@app.route('/')
def home():
    return render_template('index.html')

In [21]:
# In your predict() function inside app.py
@app.route('/predict', methods=['POST'])
def predict():
    article = request.form['article']

    results, consensus = predict_single_article(article, return_detailed=True)

    # Add description for each model
    descriptions = {
        "XGBoost": "An optimized gradient boosting library designed for high performance.",
        "Random Forest": "An ensemble of decision trees trained on random subsets of data.",
        "LightGBM": "A fast, gradient-boosting framework using tree-based learning algorithms.",
        "Logistic Regression": "A simple linear model for binary classification problems."
    }

    return render_template('index.html', results=results, consensus=consensus, descriptions=descriptions)

def predict_single_article(article, return_detailed=False):
    article_vectorized = vectorizer.transform([article])

    model_results = {}

    for name, model in models.items():
        prob = model.predict_proba(article_vectorized)[0]
        prediction = model.predict(article_vectorized)[0]
        confidence = max(prob) * 100
        label = "FAKE" if prediction == 1 else "REAL"
        model_results[name] = {
            "label": label,
            "confidence": confidence
        }

    # Sort by confidence descending
    model_results = dict(sorted(model_results.items(), key=lambda item: item[1]['confidence'], reverse=True))

    # Calculate consensus
    votes = [info['label'] for info in model_results.values()]
    consensus_label = "FAKE" if votes.count("FAKE") >= 3 else "REAL"

    if return_detailed:
        return model_results, consensus_label
    else:
        return {
            "article": article,
            "results": model_results,
            "consensus": consensus_label
        }

if __name__ == "__main__":
    if not os.path.exists(app.config['UPLOAD_FOLDER']):
        os.makedirs(app.config['UPLOAD_FOLDER'])
    app.run(debug=True, use_reloader=False)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Apr/2025 21:32:57] "GET / HTTP/1.1" 200 -
c:\Users\OMEN\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
127.0.0.1 - - [28/Apr/2025 21:33:10] "POST /predict HTTP/1.1" 200 -


In [ ]:
# #Define the route to be home 
# #The decorator below links the relative route of the URL to the function it is dec
# #Here, home function is with '/', our root directory.
# # Running the app sends us to index.html.
# # Note that the render_template means it looks for the file in the templates folder. 
# #use the route() decorator to tell Flask what URL should trigger our function.
# @app.route('/predict', methods=['POST'])
# def predict():
#     if model is None or vectorizer is None:
#         return render_template('index.html', prediction_text='Model not loaded correctly.')

#     try:
#         article_text = request.form['article']
#         processed_text = preprocess(article_text)
#         features = vectorizer.transform([processed_text])
#         prediction = model.predict(features)

#         output = "REAL" if prediction[0] == 1 else "FAKE"
#         return render_template('index.html', prediction_text=f'This news is {output}')
#     except Exception as e:
#         print(f"Error during prediction: {e}")
#         return render_template('index.html', prediction_text='Error during prediction.')


# if __name__ == "__main__":
#     app.run(debug=True, use_reloader=False)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [27/Apr/2025 12:06:45] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2025 12:06:49] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [27/Apr/2025 12:06:55] "POST /predict HTTP/1.1" 200 -
